1. Napisz program P L-T RUE?(S, m), który zwraca wartość prawda wtedy i

tylko wtedy, gdy zdanie S jest prawdziwe w przypisaniu m (gdzie m przyp-
isuje wartość 0 albo 1 każdemu symbolowi w S). Rozważmy, że S ma co

najwyżej dwa spójniki logiczne. Pokaż, że program działa przy zastosowa-
niu przykładu zdania i przypisaniu. Ten program można napisać w sposób

rekurencyjny, zgodnie z interpretacją spójników.

In [6]:
def pl_true(S, m):
    if isinstance(S, str):
        return m[S]
    
    if S[0] == 'NOT':
        return 1 - pl_true(S[1], m)
    
    if S[0] == 'AND':
        return min(pl_true(S[1], m), pl_true(S[2], m))
    
    if S[0] == 'OR':
        return max(pl_true(S[1], m), pl_true(S[2], m))
    
    if S[0] == 'IMPLIES':
        a = pl_true(S[1], m)
        b = pl_true(S[2], m)
        return max(1 - a, b)

    raise ValueError("Nieznany operator lub zła forma zdania.")


In [7]:
S1 = ('AND', 'A', 'B')
m1 = {'A': 1, 'B': 0}
print(pl_true(S1, m1))  

S2 = ('OR', 'A', ('AND', 'B', 'A'))
m2 = {'A': 1, 'B': 0}
print(pl_true(S2, m2))  

S3 = ('IMPLIES', 'A', 'B')
m3 = {'A': 1, 'B': 1}
print(pl_true(S3, m3))  

S4 = ('NOT', ('AND', 'A', 'B'))
m4 = {'A': 1, 'B': 1}
print(pl_true(S4, m4)) 

0
1
1
0


3.2 Rozważ przykład omówiony na slajdzie 22 pliku SI-W8. Napisz program, używając reguły rezolucji, aby sprawdzić, czy KB |= α.

In [14]:
def negate_literal(literal):
    return literal[1:] if literal.startswith("~") else "~" + literal

def pl_resolve(ci, cj):
    resolvents = set()
    for l in ci:
        if negate_literal(l) in cj:
            new_clause = (ci - {l}) | (cj - {negate_literal(l)})
            resolvents.add(frozenset(new_clause))
    return resolvents

def pl_resolution(KB, alpha):
    from itertools import combinations

    # Klauzule z KB
    clauses = set(frozenset(clause) for clause in KB)
    # Negacja alpha
    neg_alpha = [{negate_literal(lit)} for lit in alpha]
    clauses |= set(frozenset(clause) for clause in neg_alpha)

    while True:
        new = set()
        for (ci, cj) in combinations(clauses, 2):
            resolvents = pl_resolve(ci, cj)
            if frozenset() in resolvents:
                return True  # sprzeczność
            new |= resolvents
        if new.issubset(clauses):
            return False  # nic nowego
        clauses |= new

KB = [
    {"~P2,1", "B1,1"},            # 1
    {"~P1,2", "B1,1"},            # 2
    {"~B1,1", "P1,2", "P2,1"},    # 3
    {"~B1,1"},                    # 4
]

alpha = ["~P1,2"]  # czyli sprawdzamy: KB ⊨ ¬P1,2

# Sprawdzenie
result = pl_resolution(KB, alpha)
print("KB ⊨ ¬P1,2 ?", result)


KB ⊨ ¬P1,2 ? True


8. Znajdź unifikator dla α = Older(Father(y), y) i β = Older(Father(x), John).

In [10]:
from collections import namedtuple

Var = namedtuple('Var', ['name'])
Func = namedtuple('Func', ['name', 'args'])

def unify(x, y, subst={}):
    if subst is None:
        return None
    elif x == y:
        return subst
    elif isinstance(x, Var):
        return unify_var(x, y, subst)
    elif isinstance(y, Var):
        return unify_var(y, x, subst)
    elif isinstance(x, Func) and isinstance(y, Func):
        if x.name != y.name or len(x.args) != len(y.args):
            return None
        for a, b in zip(x.args, y.args):
            subst = unify(a, b, subst)
            if subst is None:
                return None
        return subst
    else:
        return None

def unify_var(var, x, subst):
    if var.name in subst:
        return unify(subst[var.name], x, subst)
    elif isinstance(x, Var) and x.name in subst:
        return unify(var, subst[x.name], subst)
    elif occurs_check(var, x, subst):
        return None 
    else:
        subst[var.name] = x
        return subst

def occurs_check(var, x, subst):
    if var == x:
        return True
    elif isinstance(x, Func):
        return any(occurs_check(var, arg, subst) for arg in x.args)
    elif isinstance(x, Var) and x.name in subst:
        return occurs_check(var, subst[x.name], subst)
    return False

def apply_subst(subst):
    changed = True
    while changed:
        changed = False
        for var in list(subst):
            val = subst[var]
            if isinstance(val, Var) and val.name in subst:
                subst[var] = subst[val.name]
                changed = True
    return subst

α = Func('Older', [Func('Father', [Var('y')]), Var('y')])

β = Func('Older', [Func('Father', [Var('x')]), 'John'])

subst = unify(α, β)

if subst:
    subst = apply_subst(subst)

print(subst)


{'y': 'John', 'x': 'John'}


5. Uzywając tabeli prawdziwości sprawdź czy (p ⇒ q) |= ((p ∧ r) ⇒ q).

In [11]:
import pandas as pd
import itertools

print("\n5. Uzywając tabeli prawdziwości sprawdź czy (p ⇒ q) |= ((p ∧ r) ⇒ q).")

values = list(itertools.product([0, 1], repeat=3))
rows = []

for p, q, r in values:
    implication1 = int((not p) or q)
    implication2 = int((not (p and r)) or q)
    rows.append([p, q, r, implication1, implication2])

df1 = pd.DataFrame(rows, columns=["p", "q", "r", "p ⇒ q", "(p ∧ r) ⇒ q"])
print(df1.to_string(index=False))




5. Uzywając tabeli prawdziwości sprawdź czy (p ⇒ q) |= ((p ∧ r) ⇒ q).
 p  q  r  p ⇒ q  (p ∧ r) ⇒ q
 0  0  0      1            1
 0  0  1      1            1
 0  1  0      1            1
 0  1  1      1            1
 1  0  0      0            1
 1  0  1      0            0
 1  1  0      1            1
 1  1  1      1            1


6. Uzywając tabeli prawdziwości znajdź CNF i DNF dla zdań w zadaniu 4

In [15]:
import pandas as pd
import itertools

print("\n6. Uzywając tabeli prawdziwości znajdź CNF i DNF dla zdań w zadaniu 4.")
print("(i) (p ⇒ q) ⇒ (¬p ⇒ ¬q)")

rows = []
for p, q in itertools.product([0, 1], repeat=2):
    imp1 = int((not p) or q)
    imp2 = int(p or (not q))
    whole = int((not imp1) or imp2)
    rows.append([p, q, imp1, imp2, whole])

df2 = pd.DataFrame(rows, columns=["p", "q", "p ⇒ q", "¬p ⇒ ¬q", "Całe zdanie"])
print(df2.to_string(index=False))

print("\nDNF = (¬p ∧ ¬q) ∨ (p ∧ ¬q) ∨ (p ∧ q)")
print("CNF = (¬p ∨ q)")

print("\n(ii) (p ⇒ q) ⇒ ((p ∧ r) ⇒ q)")

rows = []
for p, q, r in itertools.product([0, 1], repeat=3):
    imp1 = int((not p) or q)
    imp2 = int((not (p and r)) or q)
    whole = int((not imp1) or imp2)
    rows.append([p, q, r, imp1, imp2, whole])

df3 = pd.DataFrame(rows, columns=["p", "q", "r", "p ⇒ q", "(p ∧ r) ⇒ q", "Całe zdanie"])
print(df3.to_string(index=False))

print("\nDNF = (¬p ∧ ¬q ∧ ¬r) ∨ (¬p ∧ ¬q ∧ r) ∨ (¬p ∧ q ∧ ¬r) ∨ (¬p ∧ q ∧ r) ∨ (p ∧ ¬q ∧ ¬r) ∨ (p ∧ ¬q ∧ r) ∨ (p ∧ q ∧ ¬r) ∨ (p ∧ q ∧ r)")
print("CNF = BRAK")


6. Uzywając tabeli prawdziwości znajdź CNF i DNF dla zdań w zadaniu 4.
(i) (p ⇒ q) ⇒ (¬p ⇒ ¬q)
 p  q  p ⇒ q  ¬p ⇒ ¬q  Całe zdanie
 0  0      1        1            1
 0  1      1        0            0
 1  0      0        1            1
 1  1      1        1            1

DNF = (¬p ∧ ¬q) ∨ (p ∧ ¬q) ∨ (p ∧ q)
CNF = (¬p ∨ q)

(ii) (p ⇒ q) ⇒ ((p ∧ r) ⇒ q)
 p  q  r  p ⇒ q  (p ∧ r) ⇒ q  Całe zdanie
 0  0  0      1            1            1
 0  0  1      1            1            1
 0  1  0      1            1            1
 0  1  1      1            1            1
 1  0  0      0            1            1
 1  0  1      0            0            1
 1  1  0      1            1            1
 1  1  1      1            1            1

DNF = (¬p ∧ ¬q ∧ ¬r) ∨ (¬p ∧ ¬q ∧ r) ∨ (¬p ∧ q ∧ ¬r) ∨ (¬p ∧ q ∧ r) ∨ (p ∧ ¬q ∧ ¬r) ∨ (p ∧ ¬q ∧ r) ∨ (p ∧ q ∧ ¬r) ∨ (p ∧ q ∧ r)
CNF = BRAK


9. Rozważmy następujący problem.
Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany.
Nikt nie kocha tego, kto zabija zwierzę.
Jack kocha wszystkie zwierzęta.
Albo Jack, albo Jola zabili kota, który ma na imię Tuna.
Sprawdźmy, czy Jola zabiła kota.
(i) Najpierw napisz wszystkie wymagane zdania w KB, a wnioski do sprawdzenia
w języku logiki pierwszego rzędu. (ii) Konwertuj wszystkie zdania w CNF za
pomocą procesu skolemizacji. (iii) Następnie zastosuj regułę rezolucji, aby
sprawdzić, czy z KB wynika wniosek.

In [13]:
from typing import List, Set, Tuple, Dict, Optional
from copy import deepcopy

def parse_literal(literal: str) -> Tuple[str, List[str]]:

    negated = literal.startswith("~")
    if negated:
        literal = literal[1:]
    pred, args = literal.split("(", 1)
    args = args.rstrip(")").split(",")
    return ("~" + pred if negated else pred), args

def build_literal(pred: str, args: List[str]) -> str:
    return f"{pred}({','.join(args)})" if not pred.startswith("~") else f"~{pred[1:]}({','.join(args)})"



def unify(lit1: str, lit2: str) -> Optional[Dict[str, str]]:
    pred1, args1 = parse_literal(lit1)
    pred2, args2 = parse_literal(lit2)
    if pred1 != negate_predicate(pred2):
        return None
    if len(args1) != len(args2):
        return None
    subst = {}
    for a1, a2 in zip(args1, args2):
        if a1 == a2:
            continue
        if is_variable(a1):
            subst[a1] = a2
        elif is_variable(a2):
            subst[a2] = a1
        else:
            return None
    return subst

def is_variable(s: str) -> bool:
    return s[0].islower()

def substitute(literal: str, subst: Dict[str, str]) -> str:
    pred, args = parse_literal(literal)
    new_args = [subst.get(arg, arg) for arg in args]
    return build_literal(pred, new_args)



def negate_predicate(p: str) -> str:
    return p[1:] if p.startswith("~") else "~" + p

def resolve(ci: Set[str], cj: Set[str]) -> List[Set[str]]:
    resolvents = []
    for li in ci:
        for lj in cj:
            theta = unify(li, lj)
            if theta is not None:
                new_clause = {substitute(l, theta) for l in (ci | cj) if l != li and l != lj}
                resolvents.append(new_clause)
    return resolvents

def pretty_clause(clause: Set[str]) -> str:
    return ' ∨ '.join(sorted(clause)) if clause else '∅'

def pl_resolution(kb: List[Set[str]], alpha: Set[str]) -> bool:
    clauses = deepcopy(kb) + [set(negate_predicate(lit) for lit in alpha)]
    new = []

    print("Początkowe klauzule:")
    for i, c in enumerate(clauses):
        print(f"C{i+1}: {pretty_clause(c)}")
    print()

    steps = 1
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]

        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)
            for r in resolvents:
                steps += 1
                if not r:
                    print("Osiągnięto sprzeczność (pusta klauzula).")
                    return True
                if r not in clauses and r not in new:
                    new.append(r)
        if not new:
            print("Nie znaleziono nowych klauzul — nie można udowodnić.")
            return False
        clauses.extend(new)



kb = [
    {"Zwierzę(Tuna)"},
    {"Kocha(Jack,x)", "~Zwierzę(x)"},
    {"~Zabił(x,y)", "~Zwierzę(y)", "~KochanyPrzez(x)"},
    {"Kocha(Jack,Tuna)"},
    {"KochanyPrzez(Jack)"},
    {"Zabił(Jack,Tuna)", "Zabił(Jola,Tuna)"}
]

alpha = {"Zabił(Jola,Tuna)"}



print("Rezolucja dla celu: Zabił(Jola, Tuna)\n")
if pl_resolution(kb, alpha):
    print("Wniosek: Jola zabiła Tuna.")
else:
    print("Nie można logicznie dowieść, że Jola zabiła kota.")

Rezolucja dla celu: Zabił(Jola, Tuna)

Początkowe klauzule:
C1: Zwierzę(Tuna)
C2: Kocha(Jack,x) ∨ ~Zwierzę(x)
C3: ~KochanyPrzez(x) ∨ ~Zabił(x,y) ∨ ~Zwierzę(y)
C4: Kocha(Jack,Tuna)
C5: KochanyPrzez(Jack)
C6: Zabił(Jack,Tuna) ∨ Zabił(Jola,Tuna)
C7: ~Zabił(Jola,Tuna)

Osiągnięto sprzeczność (pusta klauzula).
Wniosek: Jola zabiła Tuna.
